This feature depends on a Catalyst PR that is not merged yet (expected within the netxt 2 weeks). If any features do not work, isntall the following branch: https://github.com/SciML/Catalyst.jl/pull/663 via

In [ ]:
Pkg.add("Calayst#spatial_SSA_support")

# Spatial simulations
A relatively recent feature, Catalyst also supports the simulation of spatial reaction networks. With the interface still being worked on, we appreciate any input!

### Deterministic, ODE, spatial simulations
We begin with spatial ODE simulations. To create a spatial model, we need to combine:
- A non-spatial reaction network model (created using the normal approach).
- A set of rules for how species move spatially (here, a set of *spatial reactions*).
- A graph (or lattice) on which the system exist.

Here, we create a spatial Brusselator model. We have linear diffusion of the component `X` (created using the `@transportation_reaction` macro). We create a square graph to simulate the system on. Finally, we combine everything into a `LatticeReactionSystem`.

In [ ]:
using Catalyst, Graphs

brusselator = @reaction_network begin
    A, ∅ → X
    1, 2X + Y → 3X
    B, X → Y
    1, X → ∅
end
tr = @transport_reaction D X
lattice = Graphs.grid([50, 50])
lrs = LatticeReactionSystem(brusselator, [tr], lattice);

Here, we model the concentration of `X` and `Y` across a *50x50* grid of compartments (could e.g. be cells). Within each compartment, we run a Brusselator simulation. In addition, we have a single spatial reaction, where `X` moves to an adjacent compartment at rate `D`. The parameters and initial conditions may vary between compartments (or, in the case of the parameter `D`, the edges connecting compartments). Here, we will use identical values for all species and parameters, except for `X`, which we will assign random values (between `0` and `10`) in all compartments.

In [ ]:
u0 = [:X => 10 * rand(nv(lattice)), :Y => 5.0]
tspan = (0.0, 100.0)
p = [:A => 1.0, :B => 4.0, :D => 0.2];

Finally, we can use the same notation as previously to bundle our model, initial conditions, time span, and parameters into a `ODEProblem` Next, we simulate the `ODEProblem` using the `QNDF` solver (which performs well for large models). 

In [ ]:
using OrdinaryDiffEq
o_prob = ODEProblem(lrs, u0, tspan, p)
o_sol = solve(o_prob, QNDF(); saveat=0.1); # saveat=0.1 ensures a smooth plot.

The plotting interface is still a work in progress and not yet available online. We will instead fetch a local version from the "spatial_plotting.jl" file.

In [ ]:
include("spatial_plotting.jl");

In [ ]:
animate_spatial_sol(o_sol,50)

### Spatial SSA simulation
Just like non-spatial ODE and Gillespie simulations are created in similar ways, spatial ODE and Gillespie simulations uses the same syntax. We modify out previous initial condition (so that the values are integers) and then create a `JumpProblem`. Note that we here use `NSM()`, one of several available specialised spatial simulators for gillespie-type simulations.

In [ ]:
using JumpProcesses    # Package for Gillespie simulations (included in DifferentialEquations.jl).
u0_ssa = [:X => rand(1:10, nv(lattice)), :Y => 5]
d_prob = DiscreteProblem(lrs, u0_ssa, (0.0,10.0), [:A => 1.0, :B => 4.0, :D => 0.2])
j_prob = JumpProblem(lrs, d_prob, NSM(); save_positions=(false,false))
j_sol = solve(j_prob, SSAStepper(); saveat=0.1);

We can now animate our output using the same syntax as previously (but giving the new plot a different name).

In [ ]:
animate_spatial_sol(j_sol, 50; outputname = "spatial_ssa_animation.mp4", framerate=100)